In [ ]:
# !python -m pip install tensorflow
# !python -m pip uninstall tensorflow --yes 

In [ ]:
import pandas as pd
import numpy as np

def normalize_feature(df):
    return df.apply(lambda column: (column - column.mean()) / column.std())

df = normalize_feature(pd.read_csv('data-3d.csv', names = ['square', 'bedrooms', 'price']))
ones = pd.DataFrame({'ones': np.ones(len(df))})
df = pd.concat([ones, df], axis = 1)
df.head()

In [ ]:
X_data = np.array(df[df.columns[0:3]])
y_data = np.array(df[df.columns[-1]]).reshape(len(df), 1)

print(X_data.shape, type(X_data))
print(y_data.shape, type(y_data))

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

alpha = 0.01
epoch = 500

X = tf.placeholder(tf.float32, X_data.shape)
y = tf.placeholder(tf.float32, y_data.shape)

W = tf.get_variable("weights", (X_data.shape[1], 1), initializer = tf.constant_initializer())

y_pred = tf.matmul(X, W)

loss_op = 1 / (2 * len(X_data)) * tf.matmul((y_pred - y), (y_pred - y), transpose_a = True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = alpha)
train_op = optimizer.minimize(loss_op)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1, epoch + 1):
        sess.run(train_op, feed_dict = {X: X_data, y: y_data})
        if i % 10 == 0:
            loss, w = sess.run([loss_op, W], feed_dict = {X: X_data, y: y_data})
            log_str = "Epoch %d \t Loss=%.4g \t Model: y = %.4gx1 + %.4gx2 + %.4g"
            print(log_str % (i, loss, w[1], w[2], w[0]))